# A step towards a reactive notebook

This is a proof-of-concept for an API that aims to automate the data flow in a notebook. It uses the IPython magic functions `onchange` to specify how variables depend on each other in a cell, and `compute` to get the desired results. The dependency resolution is automatically taken care of, and the computations only occur if needed.

In [1]:
%load_ext cellflow

In [2]:
%cellflow_configure -v

Verbose mode enabled


In [3]:
# initialization

a = 2

Any change in the value of `a` will cause `b` to be re-computed:

In [4]:
%%onchange a -> b
b = a - 1

Any change in the value of `a` or `b` will cause `c` and `d` to be re-computed:

In [5]:
%%onchange a, b -> c, d
c = a + b
d = a - b

Until now no computation did actually take place. It has to be explicitely asked for through the `compute` magic function, which will figure out the optimal way to compute the results:

In [6]:
%%compute c, d
print()
print(f'c = {c}')
print(f'd = {d}')

The data flow consists of the following paths:
a -> c
a -> d
a -> b -> c
a -> b -> d

Looking at target variable c in path: a -> c
Variable c is also in path: a -> b -> c
And other variables have to be computed first

Looking at target variable d in path: a -> d
Variable d is also in path: a -> b -> d
And other variables have to be computed first

Looking at target variable b in path: a -> b -> c
Variable b is also in path: a -> b -> d
Which doesn't prevent computing it
Variable a has changed
Computing:
b = a - 1

Looking at target variable b in path: a -> b -> d
No change in dependencies

Looking at target variable c in path: a -> c
Variable c is also in path: b -> c
Which doesn't prevent computing it
Variable a has changed
Computing:
c = a + b
d = a - b

Looking at target variable d in path: a -> d
Variable d is also in path: b -> d
Which doesn't prevent computing it
No change in dependencies

Looking at target variable c in path: b -> c
No change in dependencies

Looking at target v

If no dependency has changed, there will actually be no computation:

In [7]:
%%compute c, d
print()
print(f'c = {c}')
print(f'd = {d}')

The data flow consists of the following paths:
a -> c
a -> d
a -> b -> c
a -> b -> d

Looking at target variable c in path: a -> c
Variable c is also in path: a -> b -> c
And other variables have to be computed first

Looking at target variable d in path: a -> d
Variable d is also in path: a -> b -> d
And other variables have to be computed first

Looking at target variable b in path: a -> b -> c
Variable b is also in path: a -> b -> d
Which doesn't prevent computing it
No change in dependencies

Looking at target variable b in path: a -> b -> d
No change in dependencies

Looking at target variable c in path: a -> c
Variable c is also in path: b -> c
Which doesn't prevent computing it
No change in dependencies

Looking at target variable d in path: a -> d
Variable d is also in path: b -> d
Which doesn't prevent computing it
No change in dependencies

Looking at target variable c in path: b -> c
No change in dependencies

Looking at target variable d in path: b -> d
No change in depende

But a change in the dependencies will cause some or all the variables to be re-computed:

In [8]:
a = 3

In [9]:
%%compute c, d
print()
print(f'c = {c}')
print(f'd = {d}')

The data flow consists of the following paths:
a -> c
a -> d
a -> b -> c
a -> b -> d

Looking at target variable c in path: a -> c
Variable c is also in path: a -> b -> c
And other variables have to be computed first

Looking at target variable d in path: a -> d
Variable d is also in path: a -> b -> d
And other variables have to be computed first

Looking at target variable b in path: a -> b -> c
Variable b is also in path: a -> b -> d
Which doesn't prevent computing it
Variable a has changed
Computing:
b = a - 1

Looking at target variable b in path: a -> b -> d
No change in dependencies

Looking at target variable c in path: a -> c
Variable c is also in path: b -> c
Which doesn't prevent computing it
Variable a has changed
Computing:
c = a + b
d = a - b

Looking at target variable d in path: a -> d
Variable d is also in path: b -> d
Which doesn't prevent computing it
No change in dependencies

Looking at target variable c in path: b -> c
No change in dependencies

Looking at target v

Again, only what is needed is re-computed:

In [10]:
%%compute c, d
print()
print(f'c = {c}')
print(f'd = {d}')

The data flow consists of the following paths:
a -> c
a -> d
a -> b -> c
a -> b -> d

Looking at target variable c in path: a -> c
Variable c is also in path: a -> b -> c
And other variables have to be computed first

Looking at target variable d in path: a -> d
Variable d is also in path: a -> b -> d
And other variables have to be computed first

Looking at target variable b in path: a -> b -> c
Variable b is also in path: a -> b -> d
Which doesn't prevent computing it
No change in dependencies

Looking at target variable b in path: a -> b -> d
No change in dependencies

Looking at target variable c in path: a -> c
Variable c is also in path: b -> c
Which doesn't prevent computing it
No change in dependencies

Looking at target variable d in path: a -> d
Variable d is also in path: b -> d
Which doesn't prevent computing it
No change in dependencies

Looking at target variable c in path: b -> c
No change in dependencies

Looking at target variable d in path: b -> d
No change in depende

The data flow can be hacked. Here we modify the intermediary variable `b` (that should automatically be computed as `b = a - 1`), which will cause the final results to be re-computed:

In [11]:
b = 10

In [12]:
%%compute c
print()
print(f'c = {c}')
print(f'd = {d}')

The data flow consists of the following paths:
a -> c
a -> b -> c

Looking at target variable c in path: a -> c
Variable c is also in path: a -> b -> c
And other variables have to be computed first

Looking at target variable b in path: a -> b -> c
No change in dependencies

Looking at target variable c in path: a -> c
Variable c is also in path: b -> c
Which doesn't prevent computing it
No change in dependencies

Looking at target variable c in path: b -> c
Variable b has changed
Computing:
c = a + b
d = a - b

All done!

c = 13
d = -7


In [13]:
%%compute d
print()
print(f'c = {c}')
print(f'd = {d}')

The data flow consists of the following paths:
a -> d
a -> b -> d

Looking at target variable d in path: a -> d
Variable d is also in path: a -> b -> d
And other variables have to be computed first

Looking at target variable b in path: a -> b -> d
No change in dependencies

Looking at target variable d in path: a -> d
Variable d is also in path: b -> d
Which doesn't prevent computing it
No change in dependencies

Looking at target variable d in path: b -> d
No change in dependencies

All done!

c = 13
d = -7
